In [231]:
import sqlalchemy as sqla 
from sqlalchemy import create_engine 
import traceback 
import glob 
import os 

import requests 
import time 
import json


from IPython.display import display


import pymysql

In [232]:
import requests
import traceback
import datetime
import time
import os

In [233]:
URI="dbbikes.cjk4ybuxtkwv.us-east-1.rds.amazonaws.com"
PORT="3306"
DB="dbbikes"
USER="cmcelduff"
PASSWORD="Tullamore1!"

In [234]:
#Connect to database
engine = create_engine("mysql+pymysql://{0}:{1}@{2}:{3}".format(USER, PASSWORD, URI, PORT), echo=True) 
connection = engine.connect()

2023-02-23 16:28:25,202 INFO sqlalchemy.engine.Engine SELECT @@sql_mode
2023-02-23 16:28:25,207 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-02-23 16:28:25,312 INFO sqlalchemy.engine.Engine SELECT @@lower_case_table_names
2023-02-23 16:28:25,314 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-02-23 16:28:25,511 INFO sqlalchemy.engine.Engine SELECT DATABASE()
2023-02-23 16:28:25,513 INFO sqlalchemy.engine.Engine [raw sql] {}


In [235]:
sql = """
CREATE DATABASE IF NOT EXISTS dublin_bikes;
"""
engine.execute(sql)

2023-02-23 16:28:27,712 INFO sqlalchemy.engine.Engine 
CREATE DATABASE IF NOT EXISTS dublin_bikes;

2023-02-23 16:28:27,713 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-02-23 16:28:27,827 INFO sqlalchemy.engine.Engine COMMIT


In [236]:
# command to check database is running
for res in engine.execute("SHOW VARIABLES;"):
    print(res)

2023-02-23 16:28:30,067 INFO sqlalchemy.engine.Engine SHOW VARIABLES;
2023-02-23 16:28:30,069 INFO sqlalchemy.engine.Engine [raw sql] {}
('activate_all_roles_on_login', 'OFF')
('admin_address', '')
('admin_port', '33062')
('admin_ssl_ca', '')
('admin_ssl_capath', '')
('admin_ssl_cert', '')
('admin_ssl_cipher', '')
('admin_ssl_crl', '')
('admin_ssl_crlpath', '')
('admin_ssl_key', '')
('admin_tls_ciphersuites', '')
('admin_tls_version', 'TLSv1.2,TLSv1.3')
('authentication_policy', '*')
('auto_generate_certs', 'ON')
('auto_increment_increment', '1')
('auto_increment_offset', '1')
('autocommit', 'OFF')
('automatic_sp_privileges', 'ON')
('avoid_temporal_upgrade', 'OFF')
('awsauthenticationplugin_max_backoff_delay', '2000')
('awsauthenticationplugin_max_retry_count', '3')
('awsauthenticationplugin_retry_delay', '250')
('back_log', '61')
('basedir', '/rdsdbbin/mysql-8.0.28.R4/')
('big_tables', 'OFF')
('bind_address', '*')
('binlog_cache_size', '32768')
('binlog_checksum', 'CRC32')
('binlog_di

In [237]:
sql_create_schema = "CREATE SCHEMA `dublin_bikes`;"
try:
    #res = engine.execute("DROP TABLE IF EXISTS station")
    res = engine.execute(sql_create_schema)
    print(res.fetchall())
except Exception as e:
    print(e)

2023-02-23 16:28:30,538 INFO sqlalchemy.engine.Engine CREATE SCHEMA `dublin_bikes`;
2023-02-23 16:28:30,540 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-02-23 16:28:30,638 INFO sqlalchemy.engine.Engine ROLLBACK
(pymysql.err.ProgrammingError) (1007, "Can't create database 'dublin_bikes'; database exists")
[SQL: CREATE SCHEMA `dublin_bikes`;]
(Background on this error at: https://sqlalche.me/e/14/f405)


In [238]:
# command to create table in mysql
sql_create_table = """
CREATE TABLE IF NOT EXISTS `dublin_bikes`.`station` (
address VARCHAR(256),
banking INTEGER,
bike_stands INTEGER,
bonus INTEGER,
contact_name VARCHAR(256),
name VARCHAR(256),
number INTEGER,
position_lat REAL,
position_lng REAL,
status VARCHAR(256)
)
"""

try:
    #res = engine.execute("DROP TABLE IF EXISTS station")
    res = engine.execute(sql_create_table)
    print(res.fetchall())
except Exception as e:
    print(e)

    # tom test 
    # tom test 2

2023-02-23 16:28:31,395 INFO sqlalchemy.engine.Engine 
CREATE TABLE IF NOT EXISTS `dublin_bikes`.`station` (
address VARCHAR(256),
banking INTEGER,
bike_stands INTEGER,
bonus INTEGER,
contact_name VARCHAR(256),
name VARCHAR(256),
number INTEGER,
position_lat REAL,
position_lng REAL,
status VARCHAR(256)
)

2023-02-23 16:28:31,398 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-02-23 16:28:31,534 INFO sqlalchemy.engine.Engine COMMIT
This result object does not return rows. It has been closed automatically.


In [239]:
sql_create_table = """
CREATE TABLE IF NOT EXISTS `dublin_bikes`.`availability` (
number INTEGER,
available_bikes INTEGER,
available_bike_stands INTEGER,
last_update INTEGER
)
"""

try:
    #res = engine.execute("DROP TABLE IF EXISTS station")
    res = engine.execute(sql_create_table)
    print(res.fetchall())
except Exception as e:
    print(e)

2023-02-23 16:28:31,940 INFO sqlalchemy.engine.Engine 
CREATE TABLE IF NOT EXISTS `dublin_bikes`.`availability` (
number INTEGER,
available_bikes INTEGER,
available_bike_stands INTEGER,
last_update INTEGER
)

2023-02-23 16:28:31,942 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-02-23 16:28:32,076 INFO sqlalchemy.engine.Engine COMMIT
This result object does not return rows. It has been closed automatically.


In [240]:
NAME="Dublin"
STATIONS="https://api.jcdecaux.com/vls/v1/stations/"
DubBike_API = "7f06972a5ed335cf697379627fd13027274927c7"

def write_to_file(text):
    date_underscores = "{:%Y_%m_%d_%H_%M_%S}".format(datetime.datetime.now())
    with open(r"data\bikes_{}".format(date_underscores).replace(" ", "_"), "w") as f:
        f.write(text)

def stations_to_db(text):
    stations = json.loads(text)
    print(type(stations), len(stations))
    for station in stations:
        print(station)
        vals = (station.get("address"),int(station.get("banking")), int(station.get("bike_stands")), int(station.get("bonus")),station.get("contract_name"), station.get("name"), station.get("number"), station.get("position").get("lat"), station.get("position").get("lng"), station.get("status"))
        engine.execute("INSERT INTO `dublin_bikes`.`station` values(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)", vals)
    return

def availability_to_db(text):
    stations = json.loads(text)
    #print(availability)
    for station in stations:
        #print(station.get("name"))
        #print(station.get("number"))
        #print(station.get("available_bikes"))
        #print(station.get("available_bike_stands"))
        #print(station.get("last_update"))
        vals = (int(station.get("number")), int(station.get("available_bikes")), int(station.get("available_bike_stands")), int(station.get("last_update")))
        engine.execute("INSERT INTO `dublin_bikes`.`availability` values(%s,%s,%s,%s)", vals)
    return
        
                                            
def main():
    print(os.path)
    r = requests.get(STATIONS, params={"apiKey": DubBike_API, "contract" : NAME})
    stations_to_db(r.text)
    while True:
        try:
            now = datetime.datetime.now()
            r = requests.get(STATIONS, params={"apiKey": DubBike_API, "contract" : NAME})
            print(r, now)
            write_to_file(r.text)
            #stations_to_db(r.text)
            availability_to_db(r.text)
            time.sleep(5*60)
        except:
            print(traceback.format_exc())
            #if engine is None:
                #pass
        return

if __name__== "__main__":
    main()

<module 'posixpath' from '/Users/conormcelduff/opt/anaconda3/envs/comp30830/lib/python3.10/posixpath.py'>
<class 'list'> 114
{'number': 42, 'contract_name': 'dublin', 'name': 'SMITHFIELD NORTH', 'address': 'Smithfield North', 'position': {'lat': 53.349562, 'lng': -6.278198}, 'banking': False, 'bonus': False, 'bike_stands': 30, 'available_bike_stands': 23, 'available_bikes': 7, 'status': 'OPEN', 'last_update': 1677169595000}
2023-02-23 16:28:32,646 INFO sqlalchemy.engine.Engine INSERT INTO `dublin_bikes`.`station` values(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)
2023-02-23 16:28:32,647 INFO sqlalchemy.engine.Engine [raw sql] ('Smithfield North', 0, 30, 0, 'dublin', 'SMITHFIELD NORTH', 42, 53.349562, -6.278198, 'OPEN')
2023-02-23 16:28:32,742 INFO sqlalchemy.engine.Engine COMMIT
{'number': 30, 'contract_name': 'dublin', 'name': 'PARNELL SQUARE NORTH', 'address': 'Parnell Square North', 'position': {'lat': 53.3537415547453, 'lng': -6.26530144781526}, 'banking': False, 'bonus': False, 'bike_stands': 

2023-02-23 16:28:38,920 INFO sqlalchemy.engine.Engine [raw sql] ('Portobello Road', 0, 30, 0, 'dublin', 'PORTOBELLO ROAD', 43, 53.330091, -6.268044, 'OPEN')
2023-02-23 16:28:39,016 INFO sqlalchemy.engine.Engine COMMIT
{'number': 31, 'contract_name': 'dublin', 'name': 'PARNELL STREET', 'address': 'Parnell Street', 'position': {'lat': 53.350929, 'lng': -6.265125}, 'banking': False, 'bonus': False, 'bike_stands': 20, 'available_bike_stands': 14, 'available_bikes': 6, 'status': 'OPEN', 'last_update': 1677169636000}
2023-02-23 16:28:39,213 INFO sqlalchemy.engine.Engine INSERT INTO `dublin_bikes`.`station` values(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)
2023-02-23 16:28:39,213 INFO sqlalchemy.engine.Engine [raw sql] ('Parnell Street', 0, 20, 0, 'dublin', 'PARNELL STREET', 31, 53.350929, -6.265125, 'OPEN')
2023-02-23 16:28:39,308 INFO sqlalchemy.engine.Engine COMMIT
{'number': 98, 'contract_name': 'dublin', 'name': 'FREDERICK STREET SOUTH', 'address': 'Frederick Street South', 'position': {'lat': 53.34

2023-02-23 16:28:42,983 INFO sqlalchemy.engine.Engine [raw sql] ('Earlsfort Terrace', 0, 30, 0, 'dublin', 'EARLSFORT TERRACE', 11, 53.334295, -6.258503, 'OPEN')
2023-02-23 16:28:43,080 INFO sqlalchemy.engine.Engine COMMIT
{'number': 17, 'contract_name': 'dublin', 'name': 'GOLDEN LANE', 'address': 'Golden Lane', 'position': {'lat': 53.340803, 'lng': -6.267732}, 'banking': False, 'bonus': False, 'bike_stands': 20, 'available_bike_stands': 19, 'available_bikes': 1, 'status': 'OPEN', 'last_update': 1677169369000}
2023-02-23 16:28:43,273 INFO sqlalchemy.engine.Engine INSERT INTO `dublin_bikes`.`station` values(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)
2023-02-23 16:28:43,275 INFO sqlalchemy.engine.Engine [raw sql] ('Golden Lane', 0, 20, 0, 'dublin', 'GOLDEN LANE', 17, 53.340803, -6.267732, 'OPEN')
2023-02-23 16:28:43,371 INFO sqlalchemy.engine.Engine COMMIT
{'number': 45, 'contract_name': 'dublin', 'name': 'DEVERELL PLACE', 'address': 'Deverell Place', 'position': {'lat': 53.351464, 'lng': -6.255265},

2023-02-23 16:28:46,824 INFO sqlalchemy.engine.Engine [raw sql] ('Lime Street', 0, 40, 0, 'dublin', 'LIME STREET', 62, 53.346026, -6.243576, 'OPEN')
2023-02-23 16:28:46,919 INFO sqlalchemy.engine.Engine COMMIT
{'number': 5, 'contract_name': 'dublin', 'name': 'CHARLEMONT PLACE', 'address': 'Charlemont Street', 'position': {'lat': 53.330662, 'lng': -6.260177}, 'banking': False, 'bonus': False, 'bike_stands': 40, 'available_bike_stands': 28, 'available_bikes': 12, 'status': 'OPEN', 'last_update': 1677169588000}
2023-02-23 16:28:47,111 INFO sqlalchemy.engine.Engine INSERT INTO `dublin_bikes`.`station` values(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)
2023-02-23 16:28:47,114 INFO sqlalchemy.engine.Engine [raw sql] ('Charlemont Street', 0, 40, 0, 'dublin', 'CHARLEMONT PLACE', 5, 53.330662, -6.260177, 'OPEN')
2023-02-23 16:28:47,208 INFO sqlalchemy.engine.Engine COMMIT
{'number': 97, 'contract_name': 'dublin', 'name': 'KILMAINHAM GAOL', 'address': 'Kilmainham Gaol', 'position': {'lat': 53.342113, 'lng': 

2023-02-23 16:28:50,859 INFO sqlalchemy.engine.Engine [raw sql] ("Sir Patrick's Dun", 0, 40, 0, 'dublin', "SIR PATRICK DUN'S", 58, 53.339218, -6.240642, 'OPEN')
2023-02-23 16:28:50,959 INFO sqlalchemy.engine.Engine COMMIT
{'number': 66, 'contract_name': 'dublin', 'name': 'NEW CENTRAL BANK', 'address': 'New Central Bank', 'position': {'lat': 53.347122, 'lng': -6.234749}, 'banking': False, 'bonus': False, 'bike_stands': 40, 'available_bike_stands': 21, 'available_bikes': 19, 'status': 'OPEN', 'last_update': 1677169555000}
2023-02-23 16:28:51,165 INFO sqlalchemy.engine.Engine INSERT INTO `dublin_bikes`.`station` values(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)
2023-02-23 16:28:51,167 INFO sqlalchemy.engine.Engine [raw sql] ('New Central Bank', 0, 40, 0, 'dublin', 'NEW CENTRAL BANK', 66, 53.347122, -6.234749, 'OPEN')
2023-02-23 16:28:51,267 INFO sqlalchemy.engine.Engine COMMIT
{'number': 104, 'contract_name': 'dublin', 'name': 'GRANGEGORMAN LOWER (CENTRAL)', 'address': 'Grangegorman Lower (Central)',

...

2023-02-23 16:29:06,629 INFO sqlalchemy.engine.Engine [raw sql] ('St James Hospital (Luas)', 0, 40, 0, 'dublin', 'ST JAMES HOSPITAL (LUAS)', 80, 53.341359, -6.292951, 'OPEN')
2023-02-23 16:29:06,723 INFO sqlalchemy.engine.Engine COMMIT
{'number': 41, 'contract_name': 'dublin', 'name': 'HARCOURT TERRACE', 'address': 'Harcourt Terrace', 'position': {'lat': 53.332763, 'lng': -6.257942}, 'banking': False, 'bonus': False, 'bike_stands': 20, 'available_bike_stands': 17, 'available_bikes': 3, 'status': 'OPEN', 'last_update': 1677169405000}
2023-02-23 16:29:06,923 INFO sqlalchemy.engine.Engine INSERT INTO `dublin_bikes`.`station` values(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)
2023-02-23 16:29:06,927 INFO sqlalchemy.engine.Engine [raw sql] ('Harcourt Terrace', 0, 20, 0, 'dublin', 'HARCOURT TERRACE', 41, 53.332763, -6.257942, 'OPEN')
2023-02-23 16:29:07,034 INFO sqlalchemy.engine.Engine COMMIT
{'number': 3, 'contract_name': 'dublin', 'name': 'BOLTON STREET', 'address': 'Bolton Street', 'position': {'lat'

2023-02-23 16:29:11,425 INFO sqlalchemy.engine.Engine COMMIT
2023-02-23 16:29:11,635 INFO sqlalchemy.engine.Engine INSERT INTO `dublin_bikes`.`availability` values(%s,%s,%s,%s)
2023-02-23 16:29:11,638 INFO sqlalchemy.engine.Engine [raw sql] (108, 15, 20, 1677169564000)
2023-02-23 16:29:11,734 INFO sqlalchemy.engine.Engine COMMIT
2023-02-23 16:29:11,959 INFO sqlalchemy.engine.Engine INSERT INTO `dublin_bikes`.`availability` values(%s,%s,%s,%s)
2023-02-23 16:29:11,961 INFO sqlalchemy.engine.Engine [raw sql] (20, 20, 10, 1677169341000)
2023-02-23 16:29:12,059 INFO sqlalchemy.engine.Engine COMMIT
2023-02-23 16:29:12,269 INFO sqlalchemy.engine.Engine INSERT INTO `dublin_bikes`.`availability` values(%s,%s,%s,%s)
2023-02-23 16:29:12,273 INFO sqlalchemy.engine.Engine [raw sql] (56, 36, 4, 1677169623000)
2023-02-23 16:29:12,371 INFO sqlalchemy.engine.Engine COMMIT
2023-02-23 16:29:12,564 INFO sqlalchemy.engine.Engine INSERT INTO `dublin_bikes`.`availability` values(%s,%s,%s,%s)
2023-02-23 16:29

2023-02-23 16:29:27,263 INFO sqlalchemy.engine.Engine [raw sql] (99, 23, 7, 1677169439000)
2023-02-23 16:29:27,362 INFO sqlalchemy.engine.Engine COMMIT
2023-02-23 16:29:27,553 INFO sqlalchemy.engine.Engine INSERT INTO `dublin_bikes`.`availability` values(%s,%s,%s,%s)
2023-02-23 16:29:27,555 INFO sqlalchemy.engine.Engine [raw sql] (9, 2, 22, 1677169150000)
2023-02-23 16:29:27,648 INFO sqlalchemy.engine.Engine COMMIT
2023-02-23 16:29:27,841 INFO sqlalchemy.engine.Engine INSERT INTO `dublin_bikes`.`availability` values(%s,%s,%s,%s)
2023-02-23 16:29:27,843 INFO sqlalchemy.engine.Engine [raw sql] (67, 32, 8, 1677169258000)
2023-02-23 16:29:27,941 INFO sqlalchemy.engine.Engine COMMIT
2023-02-23 16:29:31,134 INFO sqlalchemy.engine.Engine INSERT INTO `dublin_bikes`.`availability` values(%s,%s,%s,%s)
2023-02-23 16:29:31,142 INFO sqlalchemy.engine.Engine [raw sql] (116, 2, 28, 1677169332000)
2023-02-23 16:29:31,237 INFO sqlalchemy.engine.Engine COMMIT
2023-02-23 16:29:31,446 INFO sqlalchemy.engi

2023-02-23 16:29:39,469 INFO sqlalchemy.engine.Engine INSERT INTO `dublin_bikes`.`availability` values(%s,%s,%s,%s)
2023-02-23 16:29:39,471 INFO sqlalchemy.engine.Engine [raw sql] (71, 7, 33, 1677169638000)
2023-02-23 16:29:39,569 INFO sqlalchemy.engine.Engine COMMIT
2023-02-23 16:29:39,761 INFO sqlalchemy.engine.Engine INSERT INTO `dublin_bikes`.`availability` values(%s,%s,%s,%s)
2023-02-23 16:29:39,762 INFO sqlalchemy.engine.Engine [raw sql] (79, 0, 27, 1677169285000)
2023-02-23 16:29:39,858 INFO sqlalchemy.engine.Engine COMMIT
2023-02-23 16:29:40,049 INFO sqlalchemy.engine.Engine INSERT INTO `dublin_bikes`.`availability` values(%s,%s,%s,%s)
2023-02-23 16:29:40,052 INFO sqlalchemy.engine.Engine [raw sql] (69, 30, 10, 1677169611000)
2023-02-23 16:29:40,149 INFO sqlalchemy.engine.Engine COMMIT
2023-02-23 16:29:40,342 INFO sqlalchemy.engine.Engine INSERT INTO `dublin_bikes`.`availability` values(%s,%s,%s,%s)
2023-02-23 16:29:40,344 INFO sqlalchemy.engine.Engine [raw sql] (25, 7, 23, 167

2023-02-23 16:29:48,428 INFO sqlalchemy.engine.Engine COMMIT
2023-02-23 16:29:48,627 INFO sqlalchemy.engine.Engine INSERT INTO `dublin_bikes`.`availability` values(%s,%s,%s,%s)
2023-02-23 16:29:48,628 INFO sqlalchemy.engine.Engine [raw sql] (109, 22, 7, 1677169241000)
2023-02-23 16:29:48,763 INFO sqlalchemy.engine.Engine COMMIT
2023-02-23 16:29:48,956 INFO sqlalchemy.engine.Engine INSERT INTO `dublin_bikes`.`availability` values(%s,%s,%s,%s)
2023-02-23 16:29:48,958 INFO sqlalchemy.engine.Engine [raw sql] (85, 3, 32, 1677169499000)
2023-02-23 16:29:49,051 INFO sqlalchemy.engine.Engine COMMIT
2023-02-23 16:29:49,246 INFO sqlalchemy.engine.Engine INSERT INTO `dublin_bikes`.`availability` values(%s,%s,%s,%s)
2023-02-23 16:29:49,247 INFO sqlalchemy.engine.Engine [raw sql] (107, 10, 30, 1677169369000)
2023-02-23 16:29:49,361 INFO sqlalchemy.engine.Engine COMMIT
2023-02-23 16:29:49,560 INFO sqlalchemy.engine.Engine INSERT INTO `dublin_bikes`.`availability` values(%s,%s,%s,%s)
2023-02-23 16:29

In [241]:
NAME="Dublin"
STATIONS="https://api.jcdecaux.com/vls/v1/stations"
r = requests.get(STATIONS, params={"contract" : NAME, "apiKey": DubBike_API})

In [242]:
r.json()

[{'number': 42,
  'contract_name': 'dublin',
  'name': 'SMITHFIELD NORTH',
  'address': 'Smithfield North',
  'position': {'lat': 53.349562, 'lng': -6.278198},
  'banking': False,
  'bonus': False,
  'bike_stands': 30,
  'available_bike_stands': 22,
  'available_bikes': 8,
  'status': 'OPEN',
  'last_update': 1677170112000},
 {'number': 30,
  'contract_name': 'dublin',
  'name': 'PARNELL SQUARE NORTH',
  'address': 'Parnell Square North',
  'position': {'lat': 53.3537415547453, 'lng': -6.26530144781526},
  'banking': False,
  'bonus': False,
  'bike_stands': 20,
  'available_bike_stands': 15,
  'available_bikes': 5,
  'status': 'OPEN',
  'last_update': 1677169702000},
 {'number': 54,
  'contract_name': 'dublin',
  'name': 'CLONMEL STREET',
  'address': 'Clonmel Street',
  'position': {'lat': 53.336021, 'lng': -6.26298},
  'banking': False,
  'bonus': False,
  'bike_stands': 33,
  'available_bike_stands': 8,
  'available_bikes': 25,
  'status': 'OPEN',
  'last_update': 1677170180000},
 

In [243]:
def availability_to_db(text):
    stations = json.loads(text)
    #print(availability)
    for station in stations:
        #print(station.get("name"))
        #print(station.get("number"))
        #print(station.get("available_bikes"))
        #print(station.get("available_bike_stands"))
        #print(station.get("last_update"))
        vals = (int(station.get("number")), int(station.get("available_bikes")), int(station.get("available_bike_stands")), int(station.get("last_update")))
        engine.execute("INSERT INTO `dublin_bikes`.`availability` values(%s,%s,%s,%s)", vals)
    return
        

In [244]:
# just testing out function works here
availability_to_db(r.text)

2023-02-23 16:37:16,321 INFO sqlalchemy.engine.Engine INSERT INTO `dublin_bikes`.`availability` values(%s,%s,%s,%s)
2023-02-23 16:37:16,334 INFO sqlalchemy.engine.Engine [raw sql] (42, 8, 22, 1677170112000)


OperationalError: (pymysql.err.OperationalError) (2013, 'Lost connection to MySQL server during query ([Errno 60] Operation timed out)')
[SQL: INSERT INTO `dublin_bikes`.`availability` values(%s,%s,%s,%s)]
[parameters: (42, 8, 22, 1677170112000)]
(Background on this error at: https://sqlalche.me/e/14/e3q8)

In [ ]:
def stations_to_db(text):
    stations = json.loads(text)
    print(type(stations), len(stations))
    for station in stations:
        print(station)
        vals = (station.get("address"),int(station.get("banking")), int(station.get("bike_stands")), int(station.get("bonus")),station.get("contract_name"), station.get("name"), station.get("number"), station.get("position").get("lat"), station.get("position").get("lng"), station.get("status"))
        engine.execute("INSERT INTO `dublin_bikes`.`station` values(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)", vals)
    return

In [ ]:
# just testing out function works here
availability_to_db(r.text)

In [ ]:
# just testing out function works here
stations_to_db(r.text)